In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.POD import PODNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My
import time

In [3]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_1000.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/POD_model')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/POD_model


In [4]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_1000.pkl


In [5]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [7]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [8]:
U_train, S_train, Vt_train = torch.linalg.svd(y_train.values.detach().squeeze().to('cpu').view(y_train.values.detach().shape[0], -1).T, full_matrices=False)

error = []
for mode_i in range(len(S_train)):
    error.append((sum(S_train[:mode_i])/sum(S_train)).numpy())
    if mode_i < 20:
        print(mode_i, '-->', error[mode_i])

0 --> 0.0
1 --> 0.9013104
2 --> 0.9483088
3 --> 0.9913872
4 --> 0.9943512
5 --> 0.99668187
6 --> 0.99855953
7 --> 0.99899775
8 --> 0.99935037
9 --> 0.99957174
10 --> 0.99974585
11 --> 0.99981904
12 --> 0.9998786
13 --> 0.9999184
14 --> 0.9999416
15 --> 0.99995726
16 --> 0.99997175
17 --> 0.99997973
18 --> 0.9999863
19 --> 0.99999046


In [9]:
num_modes = 13

start_time = time.time()

U_train, S_train, Vt_train = torch.linalg.svd(y_train.values.detach().squeeze().to('cpu').view(y_train.values.detach().shape[0], -1).T, full_matrices=False)

U_reduced_train = U_train[:, :num_modes]
S_reduced_train = S_train[:num_modes]
Vt_reduced_train = Vt_train[:num_modes, :]

POD_base = torch.mm(U_reduced_train, torch.diag(S_reduced_train)).to(DEVICE)
# y_train = Vt_reduced_train.T

end_time = time.time()

print(f"Tiempo de ejecución: {end_time - start_time:.6f} segundos")

Tiempo de ejecución: 0.019629 segundos


In [10]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, num_modes]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [11]:
# Load model and the optimizer
model = PODNonlinearModel(input_shape, predictive_layers, POD_base, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 3.460e+09, Test loss: 1.675e+09, MSE(e): 3.410e+02, MSE(pi1): 4.040e+03, MSE(pi2): 1.359e+02, MSE(pi3): 8.631e+01
Epoch 100, Train loss: 4.206e+06, Test loss: 4.721e+06, MSE(e): 4.108e-01, MSE(pi1): 8.381e+00, MSE(pi2): 2.934e-01, MSE(pi3): 1.441e-01
Epoch 200, Train loss: 3.398e+06, Test loss: 3.890e+06, MSE(e): 3.354e-01, MSE(pi1): 3.123e+00, MSE(pi2): 2.413e-01, MSE(pi3): 1.225e-01
Epoch 300, Train loss: 1.177e+06, Test loss: 1.615e+06, MSE(e): 1.153e-01, MSE(pi1): 1.421e+00, MSE(pi2): 8.267e-02, MSE(pi3): 9.623e-02
Epoch 400, Train loss: 1.899e+05, Test loss: 2.377e+05, MSE(e): 1.791e-02, MSE(pi1): 3.074e-01, MSE(pi2): 1.116e-02, MSE(pi3): 7.701e-02
Epoch 500, Train loss: 1.395e+05, Test loss: 2.003e+05, MSE(e): 1.313e-02, MSE(pi1): 1.216e-01, MSE(pi2): 8.314e-03, MSE(pi3): 7.031e-02
Epoch 600, Train loss: 8.185e+04, Test loss: 1.638e+05, MSE(e): 7.435e-03, MSE(pi1): 1.107e-01, MSE(pi2): 4.728e-03, MSE(pi3): 6.389e-02
Epoch 700, 

In [12]:
# Parametros de entrenamiento
start_epoch = 9000
n_epochs = 100000

batch_size = 64
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 9000.
Epoch 9000, Train loss: 5.559e+03, Test loss: 1.590e+04, MSE(e): 5.442e-04, MSE(pi1): 3.397e-03, MSE(pi2): 3.075e-04, MSE(pi3): 8.206e-04
Epoch 9100, Train loss: 2.247e+03, Test loss: 1.661e+04, MSE(e): 2.139e-04, MSE(pi1): 2.850e-03, MSE(pi2): 1.610e-04, MSE(pi3): 7.891e-04
Epoch 9200, Train loss: 2.209e+03, Test loss: 1.658e+04, MSE(e): 2.102e-04, MSE(pi1): 2.838e-03, MSE(pi2): 1.590e-04, MSE(pi3): 7.849e-04
Epoch 9300, Train loss: 2.180e+03, Test loss: 1.650e+04, MSE(e): 2.073e-04, MSE(pi1): 2.825e-03, MSE(pi2): 1.571e-04, MSE(pi3): 7.831e-04
Epoch 9400, Train loss: 2.164e+03, Test loss: 1.638e+04, MSE(e): 2.058e-04, MSE(pi1): 2.821e-03, MSE(pi2): 1.558e-04, MSE(pi3): 7.817e-04
Epoch 9500, Train loss: 2.175e+03, Test loss: 1.621e+04, MSE(e): 2.069e-04, MSE(pi1): 2.842e-03, MSE(pi2): 1.560e-04, MSE(pi3): 7.783e-04
Epoch 9600, Train loss: 2.257e+03, Test loss: 1.604e+04, MSE(e): 2.151e-04, MSE(pi1): 2.895e-03, MSE(pi2): 1.598e-04, MSE(p